In [ ]:
#Extraction of features from freezed (non-trainable) CNN layers

#import keras,numpy,os libraries

import numpy as np
import keras
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

#reading images from specified path,converting it to numpy array with images size of (224,224,3)
path = 'C:/Users/Nagaraj G/Desktop/DR_Reference/data/sample/'
img_name = os.listdir(path)
img_arr = np.zeros(shape=(10,224,224,3))
i = 0
for img in img_name:
    x = image.load_img(path+img, target_size=(224, 224,3))
    x = image.img_to_array(x)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    img_arr[i] = x
    i+=1

#loading weights without fc and softmax layers for Transfer learning.
model = VGG16(include_top = False, weights = 'imagenet',input_shape = (224,224,3))
flatten_model = Sequential()
flatten_model.add(model)
#flatten the last max pool layer outputs (last CNN layer output)
flatten_model.add(Flatten())

#extract features (convolutional autoencoder)
features_input = flatten_model.predict(img_arr)
#saving CNN outputs in a local file on disk to load later
print("Extracted features shape ",features_input.shape)
np.save("imagedata",features_input)

In [ ]:
#Training classifier network

#import keras,numpy,os, matplotlib libraries
import numpy as np
import keras
import os
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16

#reloading CNN outputs compressed image data  from file
img_arr = np.load("imagedata.npy")

#Converting labels to categorial one hot encoding
labels = keras.utils.to_categorical(np.random.randint(5, size=(10, 1)), num_classes=5)

#Adding classifier neural network (Dense + Softmax)
new_model = Sequential()
#add a fc1 layer with 8 output neurons and a dropout layer1
new_model.add(Dense(units = 8, activation = 'relu', kernel_initializer='uniform'))
new_model.add(Dropout(rate = 0.5, noise_shape=None, seed=None))
#add a fc2 layer with 8 output neurons and a dropout layer2
new_model.add(Dense(units = 8, activation = 'relu', kernel_initializer='uniform'))
new_model.add(Dropout(rate = 0.5, noise_shape=None, seed=None))
#adding a softmax layer with 5 classes
new_model.add(Dense(units = 5))
new_model.add(keras.layers.Activation('softmax'))
#configures the model for training
sgd = SGD(lr=0.001, decay=0.0, momentum=0.9,nesterov=True)
new_model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics=['accuracy'] )

#train the model for given number of epochs
training_history = {}
try:
    training_history = new_model.fit(x = features_input, y = labels, batch_size = 2, epochs = 2, verbose = 2, validation_split = 0.2)
except Exception as error:
    print(error)
finally:
    print(training_history.history)  #validation accuracy and loss, training accuracy and loss
    new_model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
    new_model.save_weights('my_model_weights.h5') #creates a HDF5 'my_model_weights.h5' weight file
#End of training and va;idation   

#plotting accuracy of training and validation
plt.plot(training_history.history['acc'])
plt.plot(training_history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#plotting loss of training and validation
plt.plot(training_history.history['loss'])
plt.plot(training_history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#Testing
#return accuracy for the test data with trained weights
try:
    loss , accuracy = new_model.evaluate(x = features_input, y = labels, batch_size= 2, verbose=1)
except Exception as error:
    print(error)
finally:
    print("loss=",loss)
    print("acc=",accuracy)


In [ ]:
#prediction of new images from trained model

#import keras,numpy,os libraries
import numpy as np
import keras
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16

img_path = 'C:/Users/Nagaraj G/Desktop/DR_Reference/data/sample/10_left.jpeg'
img_arr =  np.zeros(shape=(1,224,224,3))
img = image.load_img(img_path, target_size=(224, 224,3))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
img_arr[0] = x

#Extract the features from image needed for classifier neural net
model = VGG16(include_top = False, weights = 'imagenet',input_shape = (224,224,3))
flatten_model = Sequential()
flatten_model.add(model)
flatten_model.add(Flatten())
features_input = flatten_model.predict(img_arr)

trainedModel = load_model('my_model.h5')
ans = trainedModel.predict(features_input)

print("Prediction = ",ans)
ans = list(ans[0])
print("sum of predictions =",sum(ans))
print("max prediction value=",max(ans))
print("class the image predicted to =",ans.index(max(ans)))
